# AIN 214 - PA1 - FALL 2023

BELOW MD CELLS CONTAIN THE QUESTIONS YOU ARE ASKED TO IMPLEMENT WITHIN THE CONTEXT OF THIS HW. PLEASE FILL IN THE CELLS FOR THE ANSWERS RIGHT BELOW THE MD CELL OF THE QUESTION. YOU CAN ADD AS MANY CELLS AS YOU WANT, BE IT CODE OR MD, SO LONG AS YOU PROVIDE AN UNDERSTANDABLE AND TRACBLE REPORTING. PLEASE ADD COMMENTS ON YOUR CODES. ALSO, FILL IN MD CELLS WHERE YOU ARE ASKED YO COMMENT ON YOUR RESULTS OR EPXLAIN YOUR REASONING. ALSO, PLEASE DO NO HESITATE TO USE THEM FOR YOUR OWN REPORTING PURPOSES. PLEASE KEEP IN MIND THAT, REPORTING IS A KEY STEP IN DATA SCIENCE.

Deadline: 03.11.2023 (23:59:59)

Submission: Submit your Jupyter Notebooks via https://submit.cs.hacettepe.edu.tr/


# Necessary Imports

In [1]:
import csv#I imported this package in order to read the annots.csv file and create a sqlite3 db.
import numpy as np
import pandas as pd
from sqlite3 import connect
import sqlite3

# Notes:
* Use pandas dataframe (df) to load the data. 
* Use numpy or pandas operations for the requested tasks unless otherwise specified. Use of naive for loops is not considered valid.
* For the db operations, use sqlite3 library in Python

# 1st Set of Questions on Boston Housing Dataset **(30 Pts)**

* Retrieved from KAGGLE
* **Dataset Path: "Data/BostonHousing.csv"**

## Q1. Basic Data Exploration **(10 Pts)**
1. Load the data into Pandas DataFrame and print the column names. **(1 pts)**
2. Display the first and last 10 rows of the dataset. **(1 pts)**
3. Display random 10 rows. **(1 pts)**
4. Display random 10 rows with a seed, ie, it outputs the same random rows everytime it is run. **(1 pts)**
5. Display every 15th row. **(1 pts)**
6. Get the summary stats of the dataset regarding each column. **(2 pts)**
7. Identify and count the number of missing values in each column. **(3 pts)**

In [ ]:
# Load the data into Pandas DataFrame and print the column names
filePath = "Data/BostonHousing.csv"
df = pd.read_csv(filePath)
df.columns

In [ ]:
# Display the first and last 10 rows of the dataset.
df.head(10)#first 10 elements
df.tail(10)#last 10 elements

In [ ]:
# Display the random 10 rows
df.sample(n=10)

In [ ]:
# sample using seed
df.sample(10, random_state=42)

In [ ]:
# Display every 15th row
df[::15]

In [ ]:
# Get the summary stats of the dataset regarding each column
df.describe(include='all')

In [ ]:
# Identify and count the number of missing values in each column
df.isna()#identifying each missing value
df.isna().sum()#count of the missing values for each column

## Q2. Data Filtering (10 pts)

1. Extract records where the rm (average number of rooms per dwelling) >= 7.   **(2 pts)**
2. Extract records located along the Charles River (chas = 1) and have a crime rate (crim) less than the median crime rate of the dataset.  **(2 pts)**
3. Identify neighborhoods where the average rm is greater than 6, the lstat (% lower status of the population) is below 10, and the medv is above the 75th percentile of the dataset. This would give neighborhoods with relatively larger homes, lower "lower-status" population percentage, and higher median values.  **(2 pts)**
4. Identify records where the dis (weighted distances to five Boston employment centers) is in the top 10% of the dataset and the nox (nitric oxides concentration) is in the bottom 10%, indicating neighborhoods that are farther from employment centers and have cleaner air.  **(2 pts)**
5. From the houses built before 1940 (AGE column), select those which fall in the top 20% in terms of nitric oxide concentrations (NOX). From this subset, retrieve the bottom 10 records with the lowest MEDV values.  **(2 pts)**

In [ ]:
# Extract records where the rm >= 7
df[df['rm'] >= 7]

In [ ]:
# Extract records located along the Charles River (chas = 1) and have a crime rate (crim) less than the median crime rate of the dataset.
condition = (df['chas'] == 1) & (df['crim'] < df['crim'].median())
df[condition]

In [ ]:
# Identify neighborhoods where the average rm is greater than 6, the lstat (% lower status of the population) is below 10, and the medv is above 
# the 75th percentile of the dataset. 
condition = (df['rm'] > 6) & (df['lstat'] < 10) & (df['medv'] > df['medv'].quantile(0.75)) 
df[condition]

In [ ]:
# Identify records where the dis (weighted distances to five Boston employment centers) is in the top 10% of the dataset and the nox 
# (nitric oxides concentration) is in the bottom 10%
condition = (df['dis'] > df['dis'].quantile(0.90)) & (df['nox'] < df['nox'].quantile(0.10))
df[condition]

In [ ]:
# From the houses built before 1940 (age column), select those which fall in the top 20% in terms of nitric oxide concentrations (nox). From this subset,
# retrieve the bottom 10 records with the lowest medv values.
condition = (df['age'] > 83) & (df['nox'] > df['nox'].quantile(0.80))#2023 - 1940 = 83/top 20% means above the 80% of the dataset
df[condition].sort_values(by='medv').head(10)#sorted by ascending order and took the top 10

## Q3. Calculate the average per capita crime rate (crim) for each category, "Low", "Medium" or "High" of pupil-teacher ratio (ptratio). **(10 Pts)**
The ptratio is categorized into
* Low if ptratio <= 15
* Medium if 15 < ptratio <= 20
* High if ptratio > 20

In [ ]:
#  Calculate the average per capita crime rate (crim) for each category, "Low", "Medium" or "High" of pupil-teacher ratio (ptratio)
#for Low category:
condition = df['ptratio'] <= 15
df[condition]['crim'].mean()

In [ ]:
#for Medium category:
condition = (df['ptratio'] <= 20) & (df['ptratio'] > 15)
df[condition]['crim'].mean()

In [ ]:
#for High category:
condition = df['ptratio'] > 20
df[condition]['crim'].mean()

# 2nd Set of Questions on the shot1.csv and shot2.csv Datasets **(30 Pts)**

* **Dataset Path: "Data/shot1.csv" and "Data/shot2.csv"**

## Q1. Data Merging and Consistency Checking **(10 Pts)**

* Combine the data from "shot1.csv" and "shot2.csv" into a single DataFrame, ensuring no redundant columns are present. Merge by the redundant column names, representing the same info with different naming.
*  After merging, perform a consistency check for any duplicated rows and provide a summary of your findings. Give the details (avg, mean, etc) of each feature.

In [2]:
# read csv s into dfs 
df_shot1 = pd.read_csv("Data/shot1.csv")
df_shot2 = pd.read_csv("Data/shot2.csv")

In [7]:
# merge dfs, removing redundant columns
merged_df = pd.merge(df_shot1, df_shot2, left_on=['GAME_ID', 'SHOT_RESULT', 'SHOT_DIST'],
                      right_on=['GAME_IDENTIFICATION', 'SHOT_RESULT', 'SHOT_DIST'], how='inner')
merged_df.drop('GAME_IDENTIFICATION', axis=1, inplace=True)
merged_df.head(10)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,SHOT_RESULT,PTS_TYPE,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,made,2,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,made,2,"Roberts, Brian",203148,2.9,1,2,jarrett jack,101127
2,21400899,"MAR 04, 2015 - BKN vs. CHA",H,L,-24,5,2,11:06,14.7,5,...,7.7,made,2,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
3,21400899,"MAR 04, 2015 - BKN vs. CHA",H,L,-24,5,2,11:06,14.7,5,...,7.7,made,2,"Roberts, Brian",203148,2.9,1,2,jarrett jack,101127
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,missed,3,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
5,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,missed,2,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
6,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,missed,2,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
7,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,missed,2,"Brown, Markel",203900,3.9,0,0,lance stephenson,202362
8,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,4,10:58,15.3,4,...,17.2,missed,2,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
9,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,4,10:58,15.3,4,...,17.2,missed,2,"Brown, Markel",203900,3.9,0,0,lance stephenson,202362


In [ ]:
# check for duplicate rows and provide a summary of the findings for each feature
merged_df[merged_df.duplicated()].describe()

In [5]:
# merge dfs, removing redundant columns

df1 = pd.read_csv('Data/shot1.csv')
df2 = pd.read_csv('Data/shot2.csv')

key1 = df1['GAME_ID'].values
key2 = df2['GAME_IDENTIFICATION'].values

sorted_ind1 = np.argsort(key1)
sorted_ind2 = np.argsort(key2)

sorted_df1 = df1.iloc[sorted_ind1]
sorted_df2 = df2.iloc[sorted_ind2]

df = pd.merge(sorted_df1, sorted_df2, left_on='GAME_ID', right_on='GAME_IDENTIFICATION')

# Remove the redundant columns
df = df.drop(['GAME_IDENTIFICATION'], axis=1)

display(df.head(10))

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST_y,PTS_TYPE,SHOT_RESULT_y,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,2.2,2,made,"Harris, Tobias",202699,1.7,1,2,anthony davis,203076
1,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,3.1,2,made,"O'Quinn, Kyle",203124,3.4,1,2,anthony davis,203076
2,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,4.7,2,missed,"O'Quinn, Kyle",203124,1.4,0,0,anthony davis,203076
3,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,7.4,2,missed,"O'Quinn, Kyle",203124,2.7,0,0,anthony davis,203076
4,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,16.4,2,made,"Vucevic, Nikola",202696,6.3,1,2,anthony davis,203076
5,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,4.6,2,made,"O'Quinn, Kyle",203124,7.7,1,2,anthony davis,203076
6,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,2.5,2,missed,"Asik, Omer",201600,2.4,0,0,nikola vucevic,202696
7,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,5.1,2,missed,"Asik, Omer",201600,1.7,0,0,nikola vucevic,202696
8,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,18.9,2,missed,"Asik, Omer",201600,5.5,0,0,nikola vucevic,202696
9,21400001,"OCT 28, 2014 - NOP vs. ORL",H,W,17,15,3,10:38,24.0,0,...,0.6,2,made,"Davis, Anthony",203076,2.0,1,2,nikola vucevic,202696


## Q2. Player’s Shooting Accuracy Analysis **(10 Pts)**
* Using the combined DataFrame from Question 1, calculate and display the shooting accuracy of each player. 
* Shooting accuracy = the ratio of successful shots (SHOT_RESULT) to total shots taken
* Display the top 5 players based on shooting accuracy

In [ ]:
#shooting accuracy of each player
grouped_data = merged_df.groupby('player_name')
total_shots = grouped_data['SHOT_RESULT'].count()        
made_shots = grouped_data['SHOT_RESULT'].apply(lambda x: (x == 'made').sum())
shot_accuracy = round(((made_shots / total_shots)*100),2)
result_df = pd.DataFrame({'player_name': total_shots.index, 'shot_accuracy_percentage': shot_accuracy}).reset_index(drop=True)
result_df

## Q3. Analyze the Relationship between shooting distance (SHOT_DIST) and shooting success (SHOT_RESULT). **(10 Pts)**
* Bin the shots into categories based on distance. **STATE AND REASON ABOUT YOUR LOGIC HERE!** (How to apply binning)
* Calculate the shooting accuracy for each bin. 
* **COMMENT** on your findinds.

In [ ]:
#Shot data categorized based on the distance(SHOT_DIST)
#long_shot > 20, 20 >= intermediate_shot > 10, close_shot <= 10
condition1 = merged_df['SHOT_DIST'] > 20
condition2 = (merged_df['SHOT_DIST'] <= 20) & (merged_df['SHOT_DIST'] > 10)
condition3 = merged_df['SHOT_DIST'] <= 10
merged_df['SHOT_CATEGORY'] = np.where(condition1, 'long_shot', np.where(condition2, 'medium_shot', 'close_shot'))
grouped = merged_df.groupby('SHOT_CATEGORY')
total_shots = grouped['SHOT_RESULT'].count()
made_shots = grouped['SHOT_RESULT'].apply(lambda x: (x == 'made').sum())
shot_accuracy = round(((made_shots / total_shots)*100),2)
result_df = pd.DataFrame({'shot_category': total_shots.index, 'shot_accuracy_percentage': shot_accuracy}).reset_index(drop=True)
result_df
#The shots are binned into categories and found the shot_accuracy's of each category. We can say that the shots taken closely are more likely
#to score and the shots taken by the long distance have the least accuracy percentage.

# 3rd Set of Questions on SQL - **using sqlite3 library!** **(40 Pts)**

* **Dataset Path: "Data/annots.csv"**

Here you are given a file: annots.csv, with the protein - gene ontology (GO) term associations/annotations. Gene ontology is a framework, representing the properties of genes and proteins. This is basically a Graph - more spesifically a directed acyclic graph (DAG), representing a level-ordered hierarchy with three subgraphs: MF, CC, and BP. Here, **each** node in the graph, which we term as **"GO Term" corresponds to a function of a protein.**

* If protein is annotated with a GO Term, then the protein has the function represented by this GO Term. 
* A protein can be and most of the time is annotated with multiple GO terms, and vice versa. 

In the annots.csv file, we have the annotation data, where each row corresponds to a protein - GO term annotation with the following columns: ProteinID,ProtSeq, GO_ID, Sub_Graph, Level. ProteinID is the specifier of the protein and the ProtSeq is the aminoacid sequence of this protein. GO_ID, Sub_Graph, and Level are properties of a GO Term. 

**Now, we want to store this data in a relational database, with the following table schemas:**

## Table 1: GoTerms:

| Column   | Type    | Key        | Description                |
|----------|---------|------------|----------------------------|
| GoID     | TEXT    | PRIMARY KEY| Gene Ontology Identifier   |
| SubGraph | TEXT    |            | Subgraph Type (BP, MF, CC) |
| Level    | INTEGER |            | Ontology level             |


## Table 2: Proteins

| Column     | Type | Key        | Description                |
|------------|------|------------|----------------------------|
| ProteinID  | TEXT | PRIMARY KEY| Unique Protein Identifier  |
| ProtSeq  | TEXT | | Amino Acid Sequence |

## Table 3: Annotations

| Column    | Type | Key                       | Description               |
|-----------|------|---------------------------|---------------------------|
| GoID      | TEXT | PRIMARY KEY, FOREIGN KEY  | Gene Ontology Identifier  |
| ProteinID | TEXT | PRIMARY KEY, FOREIGN KEY  | Unique Protein Identifier |


## Q1. Read the csv file and create this DB  **(10 Pts)**

In [2]:
# read file into db and create a db using sqlite3
datasetPath = "Data/annots.csv"
conn = sqlite3.connect("annots.db")
cursor = conn.cursor()

# Create tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS GoTerms (
        GoID TEXT PRIMARY KEY,
        SubGraph TEXT,
        Level INTEGER
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Proteins (
        ProteinID TEXT PRIMARY KEY,
        ProtSeq TEXT
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Annotations (
        GoID TEXT,
        ProteinID TEXT,
        PRIMARY KEY (GoID, ProteinID),
        FOREIGN KEY (GoID) REFERENCES GoTerms(GoID),
        FOREIGN KEY (ProteinID) REFERENCES Proteins(ProteinID)
    )
''')
conn.commit()
# Import data from the CSV file
try:
    with open(datasetPath, "r") as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            cursor.execute("INSERT OR IGNORE INTO GoTerms (GoID, SubGraph, Level) VALUES (?, ?, ?)", (row[2], row[3], row[4]))
            cursor.execute("INSERT OR IGNORE INTO Proteins (ProteinID, ProtSeq) VALUES (?, ?)", (row[0], row[1]))
            cursor.execute("INSERT OR IGNORE INTO Annotations (GoID, ProteinID) VALUES (?, ?)", (row[2], row[0]))
    conn.commit()
except Exception as e:
    print("Error:", e)
finally:
    conn.close()

## Q2. Define a function to extract all GO terms associated with a given amino acid sequence (ProtSeq), ensuring that the GO terms satisfy the following criteria: they are annotated with at least 50 proteins in MF and CC subgraphs, and 100 proteins in the BP subgraph.. **(10 Pts)**

In [56]:
prot_seq = "VDDIQEKMNKELGCIKVTFPGADGQGEDACLKDIPVSLISTYFARTAWTDIIRV" #  example protein sequence
#I actually couldn't find any prot_seq that satisfies the conditions you gave or maybe I misunderstood the question, it is probably the case :( 

# function to extract the annotated GO terms with the above specs
def extract_go_terms(prot_seq):
    conn = sqlite3.connect("annots.db")
    cursor = conn.cursor()
    
    results = []
    MF_count = 0#MF sub graph count should be at least 50
    CC_count = 0#CC sub graph count should be at least 50
    BP_count = 0#BP sub graph count should be at least 100
    
    try:
        cursor.execute('''
                SELECT G.GoID, G.SubGraph
                FROM GoTerms G, Annotations A, Proteins P
                WHERE G.GoID = A.GoID AND A.ProteinID = P.ProteinID
                AND P.ProtSeq = ?
                GROUP BY G.GoID
                HAVING (G.SubGraph = 'MF' OR G.SubGraph = 'CC') AND COUNT(DISTINCT P.ProteinID) >= 50
            OR (G.SubGraph = 'BP') AND COUNT(DISTINCT P.ProteinID) >= 100
        ''', (prot_seq,))
        
        go_terms = cursor.fetchall()
        results = [row[0] for row in go_terms]
        
        #this is a solution with for loop but it doesn't support the usage of group by and having keyword
        '''for go_id, sub_graph in go_terms:
            if(sub_graph == "MF"):
                MF_count += 1
            elif(sub_graph == "CC"):
                CC_count += 1
            elif(sub_graph == "BP"):
                BP_count += 1
        
        for go_id, sub_graph in go_terms:
            if(sub_graph == "MF" and MF_count >= 50) or (sub_graph == "CC" and CC_count >= 50) or (sub_graph == "BP" and BP_count >= 100):
                results.append(go_id)'''
        
    except Exception as e:
        print("Error:", e)
    finally:
        conn.close()
        
    return results

extract_go_terms(prot_seq)

[]

## Q3. Print the average number of annotations a GO terms has for each level in each subgraph (MF, CC, and BP) **(10 Pts)**

In [16]:
# Print the avg # of annots
conn = sqlite3.connect("annots.db")
cursor = conn.cursor()
try:
    cursor.execute('''
    SELECT G.Level, G.GoID, avg(count(distinct G.GoID)) 
    FROM Annotations A, GoTerms G
    WHERE A.GoID = G.GoID
    GROUP BY G.Level
    
    ''')
    
    print(cursor.fetchall())
except Exception as e:
    print("Error:", e)
finally:
    conn.close()
    

Error: no such column: SubGraph


## Q4. Identify the ProteinID with the longest ProtSeq within the Proteins table. Modify its ProtSeq to half its original length by trimming the sequence, update the ProteinID to "trimmed", and display the updated ProteinID and ProtSeq with its GO Term annotations. **(10 Pts)**

In [3]:
# find and trim the longest sequence, displaying the updated ProtID and ProtSeq, and its annotated GO Terms
conn = sqlite3.connect("annots.db")
cursor = conn.cursor()

#ordering the protseqs by length
cursor.execute('''
            SELECT ProteinID, ProtSeq 
            FROM Proteins 
            ORDER BY LENGTH(ProtSeq) DESC LIMIT 1''')

result = cursor.fetchone()
id_of_the_longest_protein = result[0]
the_longest_protseq = result[1]

# trimming the protseq
half_length = len(the_longest_protseq) // 2
trimmed_protseq = the_longest_protseq[:half_length]

new_protein_id = "trimmed"
# Insert the new row with the trimmed protein and the new ProteinID
cursor.execute("INSERT INTO Proteins (ProteinID, ProtSeq) VALUES (?, ?)", (new_protein_id, trimmed_protseq))
conn.commit()

#Copy the GO Term annotations from the old row to the new row
cursor.execute("INSERT INTO Annotations (GoID, ProteinID) SELECT GoID, ? FROM Annotations WHERE ProteinID = ?", (new_protein_id, id_of_the_longest_protein))
conn.commit()

#Delete the old row
cursor.execute("DELETE FROM Proteins WHERE ProteinID = ?", (id_of_the_longest_protein,))
conn.commit()

#since ProteinID is a primary key in this database, I wanted to add another column with the specified values and delete the row has the 
#trimmed ProtSeq (the old record) , but it didn't seem to work properly :(

cursor.execute("""
SELECT p.ProteinID, p.ProtSeq, a.GoID
FROM Proteins p, Annotations a
WHERE p.ProteinID = a.ProteinID
AND p.ProteinID = "trimmed"
""")
result = cursor.fetchall()
result

IntegrityError: UNIQUE constraint failed: Proteins.ProteinID

# PLAGIARISM
All work on assignments must be done individually. You are encouraged to discuss
the given assignments with your classmates, but these discussions should be carried out
in an abstract way. That is, discussions related to a particular solution to a specific prob-
lem (either in actual code or in pseudocode) will not be tolerated. In short, turning
in someone else’s work (including work available on the internet), in whole or in part, as
your own will be considered as a violation of academic integrity. Please note that the
former conditions also hold for the material attained using AI tools, including ChatGPT,
GitHub Copilot, etc.